## PurpleAir

In [3]:
import requests
import folium
import pandas as pd

In [4]:
endpoint = "https://www.purpleair.com/json"

In [5]:
response = requests.get(endpoint).json() # get all sensor data

In [6]:
response

{'baseVersion': '7',
 'mapVersion': '0.19',
 'mapVersionString': '',
 'results': [{'AGE': 1,
   'DEVICE_LOCATIONTYPE': 'outside',
   'Hidden': 'false',
   'ID': 14633,
   'Label': ' Hazelwood canary ',
   'LastSeen': 1600730455,
   'Lat': 37.275561,
   'Lon': -121.964134,
   'PM2_5Value': '23.6',
   'Stats': '{"v":23.6,"v1":28.07,"v2":30.89,"v3":32.83,"v4":34.0,"v5":25.61,"v6":34.93,"pm":23.6,"lastModified":1600730454958,"timeSinceModified":120080}',
   'THINGSPEAK_PRIMARY_ID': '559921',
   'THINGSPEAK_PRIMARY_ID_READ_KEY': 'CU4BQZZ38WO5UJ4C',
   'THINGSPEAK_SECONDARY_ID': '559922',
   'THINGSPEAK_SECONDARY_ID_READ_KEY': 'D0YNZ1LM59LL49VQ',
   'Type': 'PMS5003+PMS5003+BME280',
   'humidity': '24',
   'isOwner': 0,
   'pressure': '1005.38',
   'temp_f': '95'},
  {'AGE': 1,
   'Hidden': 'false',
   'ID': 14634,
   'Label': ' Hazelwood canary  B',
   'LastSeen': 1600730455,
   'Lat': 37.275561,
   'Lon': -121.964134,
   'PM2_5Value': '23.16',
   'ParentID': 14633,
   'Stats': '{"v":23.16,

In [7]:
# ROUGH boundaries of LA County
north_west = {'Lat': 34.814597, 'Lon': -118.938110}
south_west = {'Lat': 33.724877, 'Lon': -118.886869}
north_east = {'Lat': 34.822724, 'Lon': -117.658021}
south_east = {'Lat': 33.698211, 'Lon': -117.690078}


los_angeles_data = []

for result in response['results']:
    lat = result.get('Lat')
    lon = result.get('Lon')
    if(lat != None and lon != None):
        if(lat <= north_west.get('Lat') and lon >= north_west.get('Lon') 
           and lat <= north_east.get('Lat') and lon <= north_east.get('Lon')
           and lat >= south_west.get('Lat') and lon >= south_west.get('Lon')
           and lat >= south_east.get('Lat') and lon <= south_east.get('Lon')):
            los_angeles_data.append(result)
       
       

In [8]:
# Formulas from functions referenced in PurpleAir documentation 
# https://docs.google.com/document/d/15ijz94dXJ-YAZLi9iZ_RaBwrZ4KtYeCy08goGBwnbCU/edit

def calcAQI(Cp, Ih, Il, BPh, BPl):
    a = (float(Ih) - Il);
    b = (float(BPh) - BPl);
    c = (float(Cp) - BPl);
    return round((a/b) * c + Il);

def pm_to_aqi(pm):
    if float(pm) < 0:
        return pm 
    if float(pm) > 1000:
        return None 
    
    if (float(pm) > 350.5):
        return calcAQI(pm, 500, 401, 500, 350.5)
    elif (float(pm) > 250.5):
        return calcAQI(pm, 400, 301, 350.4, 250.5)
    elif (float(pm) > 150.5):
        return calcAQI(pm, 300, 201, 250.4, 150.5)
    elif (float(pm) > 55.5):
        return calcAQI(pm, 200, 151, 150.4, 55.5)
    elif (float(pm) > 35.5):
        return calcAQI(pm, 150, 101, 55.4, 35.5)
    elif (float(pm) > 12.1):
        return calcAQI(pm, 100, 51, 35.4, 12.1)
    elif (float(pm) >= 0):
        return calcAQI(pm, 50, 0, 12, 0)
    else:
        return None;

 


In [9]:
m = folium.Map(
    location=[34.1140002,-118.2643563],
    zoom_start=10
    )

In [10]:
aqi_color_values = {'Good': '#65d683', 'Moderate': '#ecf27e', 'UnhealtySensitive': '#f2cd7e', 'Unhealthy': '#f0583a', 'VeryUnhealthy': '#4b1d59', 'Hazardous': '#611d3c'}

for data in los_angeles_data:
    if (data.get('PM2_5Value') != None):
        aqi_val = pm_to_aqi(data.get('PM2_5Value'))
        if(aqi_val != None): 
            aqi_val = float(aqi_val)
            if (aqi_val >= 0 and aqi_val <= 50):
                color = aqi_color_values.get('Good')
            elif (aqi_val >= 51 and aqi_val <= 100):
                color = aqi_color_values.get('Moderate')
            elif (aqi_val >= 101 and aqi_val <= 150):
                color = aqi_color_values.get('UnhealtySensitive')
            elif (aqi_val >= 151 and aqi_val <= 200):
                color = aqi_color_values.get('Unhealthy')
            elif (aqi_val >= 201 and aqi_val <= 300):
                color = aqi_color_values.get('VeryUnhealthy')
            else:
                color = aqi_color_values.get('Hazardous')
            
            folium.CircleMarker(
                location=[data['Lat'], data['Lon']],
                radius=30,
                popup=f'PMD 2.5 AQI {aqi_val}',
                color=color,
                fill=True,
                fill_color=color
            ).add_to(m)

In [11]:
asthma_df = pd.read_excel('C:/Users/justi.JUSTINE-SURFACE/Desktop/Air Pollution vs Healthcare/M6_MedicalConditions_MED11ASTH.xlsx')
asthma_df

,HealthDistricts,Percent,EstimatedNumber,Latitude,Longitude
0,Alhambra,0.085,24000,34.085528,-118.171527
1,Antelope Valley,0.174,50000,34.751379,118.256675
2,Bellflower,0.103,28000,33.888048,-118.146550
3,Central,0.082,24000,34.084739,-118.329608
4,Compton,0.144,29000,33.893286,-118.239320
5,East LA,0.070,11000,34.035200,-118.177145
6,East Valley,0.169,61000,34.194163,-118.448516
7,El Monte,0.098,33000,34.071204,-118.052706
8,Foothill,0.114,28000,34.267154,-118.337817
9,Glendale,0.108,31000,34.193063,-118.279946


In [12]:
asthma_avg_percent = pd.to_numeric(asthma_df['Percent']).mean()
print(f'asthma average by percent {asthma_avg_percent}')

asthma average by percent 0.11257692307692307


In [15]:
for index, row in asthma_df.iterrows():
    color = ''
    print(row)
    percent = row['Percent']
    location = row['HealthDistricts']
    
    if(percent >= 0 and percent <= asthma_avg_percent):
        color = '#c2ae2f'
    elif(percent >= asthma_avg_percent and percent <= asthma_avg_percent + (asthma_avg_percent / 2)):
        color = '#d48c3f'
    else:
        color = '#e83b35'
           
    folium.CircleMarker(
            location=[row['Latitude'], row['Longitude']],
            radius=70,
            popup=f'{location}: percent of people with asthma {percent}',
            color=color,
            fill=True,
            fill_color=color
        ).add_to(m)

HealthDistricts    Alhambra
Percent               0.085
EstimatedNumber       24000
Latitude            34.0855
Longitude          -118.172
Name: 0, dtype: object
HealthDistricts    Antelope Valley
Percent                      0.174
EstimatedNumber              50000
Latitude                   34.7514
Longitude                  118.257
Name: 1, dtype: object
HealthDistricts    Bellflower
Percent                 0.103
EstimatedNumber         28000
Latitude               33.888
Longitude            -118.147
Name: 2, dtype: object
HealthDistricts    Central
Percent              0.082
EstimatedNumber      24000
Latitude           34.0847
Longitude          -118.33
Name: 3, dtype: object
HealthDistricts    Compton
Percent              0.144
EstimatedNumber      29000
Latitude           33.8933
Longitude         -118.239
Name: 4, dtype: object
HealthDistricts    East LA
Percent               0.07
EstimatedNumber      11000
Latitude           34.0352
Longitude         -118.177
Name: 5, dtype:

In [16]:
m